## Libraries

In [1]:
import pulp as lp
import random

## Src

In [2]:
def generate_vrptw_parameters(
    num_vehicles,
    num_customers,
    cost_range=(10, 50),
    time_range=(5, 30),
    demand_range=(5, 15),
    capacity=20,
    time_window_range=(0, 50),
    large_constant=1000,
):
    """
    Generates random VRPTW parameters for a given number of vehicles and customers.

    Parameters:
    - num_vehicles (int): Number of vehicles.
    - num_customers (int): Number of customers (not including depots).
    - cost_range (tuple): Range for travel costs between nodes.
    - time_range (tuple): Range for travel times between nodes.
    - demand_range (tuple): Range for customer demands.
    - capacity (int): Capacity of each vehicle.
    - time_window_range (tuple): Range for time windows (start and end times).
    - large_constant (int): Large constant for constraints.

    Returns:
    - dict: A dictionary containing all sets and parameters.
    """
    # Sets
    V = list(range(1, num_vehicles + 1))  # Vehicles labeled from 1 to num_vehicles
    N = list(
        range(num_customers + 2)
    )  # Nodes labeled from 0 to num_customers+1 (including depots)
    C = list(range(1, num_customers + 1))  # Customers labeled from 1 to num_customers

    # Parameters
    cij = {(i, j): random.randint(*cost_range) for i in N for j in N if i != j}
    tij = {(i, j): random.randint(*time_range) for i in N for j in N if i != j}
    di = {i: random.randint(*demand_range) for i in C}  # Customer demands
    q = capacity  # Vehicle capacity
    a = {i: random.randint(*time_window_range) for i in N}  # Start of time window
    b = {
        i: random.randint(a[i] + 1, time_window_range[1]) for i in N
    }  # End of time window, after start time
    K = large_constant  # Large constant for time window constraints

    # Ensuring depot time windows
    a[0], b[0] = 0, time_window_range[1]  # Starting depot time window
    a[num_customers + 1], b[num_customers + 1] = (
        0,
        time_window_range[1],
    )  # Returning depot time window

    return {
        "V": V,
        "N": N,
        "C": C,
        "cij": cij,
        "tij": tij,
        "di": di,
        "q": q,
        "a": a,
        "b": b,
        "K": K,
    }

## Constants

In [3]:
n = 3  # num of customers
num_vehicles = 2
parameters = generate_vrptw_parameters(num_vehicles=num_vehicles, num_customers=n)
print(parameters)
V, N, C, cij, tij, di, q, a, b, K = parameters.values()

{'V': [1, 2], 'N': [0, 1, 2, 3, 4], 'C': [1, 2, 3], 'cij': {(0, 1): 44, (0, 2): 43, (0, 3): 10, (0, 4): 49, (1, 0): 28, (1, 2): 34, (1, 3): 38, (1, 4): 44, (2, 0): 46, (2, 1): 19, (2, 3): 27, (2, 4): 18, (3, 0): 11, (3, 1): 10, (3, 2): 14, (3, 4): 34, (4, 0): 22, (4, 1): 27, (4, 2): 47, (4, 3): 23}, 'tij': {(0, 1): 24, (0, 2): 28, (0, 3): 8, (0, 4): 12, (1, 0): 18, (1, 2): 13, (1, 3): 5, (1, 4): 25, (2, 0): 30, (2, 1): 13, (2, 3): 13, (2, 4): 13, (3, 0): 23, (3, 1): 10, (3, 2): 14, (3, 4): 21, (4, 0): 17, (4, 1): 21, (4, 2): 30, (4, 3): 9}, 'di': {1: 8, 2: 10, 3: 14}, 'q': 20, 'a': {0: 0, 1: 35, 2: 21, 3: 22, 4: 0}, 'b': {0: 50, 1: 39, 2: 34, 3: 48, 4: 50}, 'K': 1000}


Modèle de minimisation de coût

In [4]:
problem = lp.LpProblem("VRPTW", lp.LpMinimize)

Dicision variables

In [5]:
x = lp.LpVariable.dicts("x", ((k, i, j) for k in V for i in N for j in N if i != j), cat='Binary')
s = lp.LpVariable.dicts("s", ((i, k) for i in N for k in V), lowBound=0, cat='Continuous')

Objective function

In [6]:
problem += lp.lpSum(cij[i, j] * x[k, i, j] for k in V for i in N for j in N if i != j)

## Constraints

Each customer is visited exactly once

In [7]:
for i in C:
    problem += lp.lpSum(x[k, i, j] for k in V for j in N if i != j) == 1


Vehicle capacity constraint

In [8]:

for k in V:
    problem += lp.lpSum(di[i] * x[k, i, j] for i in C for j in N if i != j) <= q


Each vehicle starts at the depot (node 0)

In [9]:
for k in V:
    problem += lp.lpSum(x[k, 0, j] for j in N if j != 0) == 1

Flow conservation for each customer

In [10]:
for k in V:
    for h in C:
        problem += lp.lpSum(x[k, i, h] for i in N if i != h) == lp.lpSum(
            x[k, h, j] for j in N if h != j
        )


Each vehicle ends at the returning depot (node n+1)

In [11]:
for k in V:
    problem += lp.lpSum(x[k, i, n + 1] for i in N if i != n + 1) == 1

Time window constraint with large constant K

In [12]:
for k in V:
    for i in N:
        for j in N:
            if i != j:
                problem += s[i, k] + tij[i, j] - K * (1 - x[k, i, j]) <= s[j, k]

Start time must be within the time window

In [13]:
for i in N:
    for k in V:
        problem += a[i] <= s[i, k]
        problem += s[i, k] <= b[i]


Binary constraint for x variables

In [14]:
for k in V:
    for i in N:
        for j in N:
            if i != j:
                problem += x[k, i, j] >= 0
                problem += x[k, i, j] <= 1

## Solution

In [15]:
problem.solve()

for var in problem.variables():
    print(f"{var.name} = {var.varValue}")

print("Solution Status:", lp.LpStatus[problem.status])
print("Objective value:", lp.value(problem.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/cuphead/Projects/VRPTW-Meta-Heuristique/venv/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/106ec39cc9564ba882875961f56da6a6-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/106ec39cc9564ba882875961f56da6a6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 160 COLUMNS
At line 613 RHS
At line 769 BOUNDS
At line 810 ENDATA
Problem MODEL has 155 rows, 50 columns and 332 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 121 - 0.00 seconds
Cgl0003I 16 fixed, 0 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0000I Cut generators found to be infeasible! (or unbounded)
Pre-processing says infeasible or unbounded
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.02

s_(0,_1) =

Get the routes for each vehicle

In [9]:
routes = {k: [] for k in V} 

for k in V:
    current_node = 0  # Start from the depot (node 0)
    while (
        current_node != n + 1
    ):  # Continue until reaching the returning depot (node n+1)
        for j in N:
            if current_node != j and lp.value(x[k, current_node, j]) == 1:
                routes[k].append((current_node, j))
                current_node = j
                break

Print the routes

In [10]:
for k in V:
    route_str = " -> ".join(str(node) for node, _ in routes[k]) + f" -> {n + 1}"
    print(f"Route for vehicle {k}: {route_str}")

Route for vehicle 1: 0 -> 2 -> 4
Route for vehicle 2: 0 -> 3 -> 1 -> 4


Display service start times for each customer

In [11]:
for k in V:
    print(f"\nService start times for vehicle {k}:")
    for i in N:
        start_time = lp.value(s[i, k])
        if (
            start_time is not None and start_time > 0
        ):  # Only print if the vehicle visits the customer
            print(f"  Customer {i}: Start time = {start_time:.2f}")


Service start times for vehicle 1:
  Customer 1: Start time = 15.00
  Customer 2: Start time = 20.00
  Customer 3: Start time = 11.00
  Customer 4: Start time = 50.00

Service start times for vehicle 2:
  Customer 1: Start time = 38.00
  Customer 2: Start time = 20.00
  Customer 3: Start time = 11.00
  Customer 4: Start time = 50.00
